In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

torch.manual_seed(777)   #random value를 잡아주는? 항상 같은 random이 나오는 걸로 설명은 되어있으나 완벽하게 이해 못함 질문1
if device == 'cuda' : 
    torch.cuda.manual_seed_all(777)

In [8]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

# data 다운 되게 만드는 보안 헤더 코드???? 질문2

In [11]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [12]:
#parameters 정의해둠 > data loader에 써야되니까 batch size등
learning_rate = 0.001
traning_epochs =15
batch_size = 100

In [15]:
data_loader = torch.utils.data.DataLoader(dataset =mnist_train,
                                         batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [5]:
class CNN(nn.Module) :
    
    def __init__(self) :
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1)
            